In [1]:
import pandas as pd
import googlemaps as gm
from geopy.geocoders import Nominatim
import os
import json
import numpy as np

In this first section, I will be taking the Menu table that Vidya has used OpenRefine to clean and cluster places and locations. We will take this one step further and harness the Google Maps places API and the Geopy API to go from place names, to latitude/longitude coordinates, and finally to regions (here as state + country or just country where state does not exist). If the places or locations are unknown, or the Google Maps API is unable to find a place, then the final entry will be "UNKNOWN".

In [2]:
gmaps = gm.Client(os.getenv('GMAPS_API'))
geolocator = Nominatim(user_agent="geoapiExercises")

In [3]:
menu_df = pd.read_csv("Menu-edited-meals-place-location-year.csv")
menu_edit = menu_df.copy()
menu_edit['loc_place'] = menu_edit['location_edited'] +', '+menu_edit['place_edited']
locations = menu_edit[(menu_edit.place_edited.str.lower()!='unknown')&(menu_edit.location_edited.str.lower()!='unknown')].loc_place.unique()

get_region() will take a location string (concatenated as location, place above) and convert it to a region where possible (returning None if not). make_region_map() will add the result of get_region() to a dictionary if it is not None.

In [4]:
def get_region(loc_string):
    results = gmaps.places(loc_string)
    if not results['results']:
        return None
    lat_long = results['results'][0]['geometry']['location']
    location = geolocator.reverse(str(lat_long['lat'])+","+str(lat_long['lng']))
    addr = location.raw['address']
    if 'state' not in addr:
        if 'country' not in addr:
            return None
        return addr['country']
    return addr['state'] +', '+ addr['country']

In [5]:
def make_region_map(locations):
    mapping = {}
    for loc in locations:
        region = get_region(loc)
        if region:
            mapping[loc] = region
    return mapping

I have commented out the actual call to make_region_map since it takes a while to run and calls the google maps api. Instead, below I save the initial run to a json file and upload that.

In [6]:
# mappings = make_region_map(locations)

In [7]:
# with open('mappings.json', 'w') as f:
#     json.dump(mappings, f)

In [8]:
f = open('mappings.json')
mappings = json.load(f)

apply_region() will run over every entry in the Menu table and either apply the corresponding region mapping from the mapping dictionary, or "UNKNOWN" if no entry exists.

In [9]:
def apply_region(df):
    loc_place = df['loc_place']
    if loc_place in mappings:
        df['region'] = mappings[loc_place]
    return df

In [10]:
menu_edit['region'] = 'UNKNOWN'
menu_edit = menu_edit.apply(apply_region, axis=1)

In [11]:
print('Number of Menus with Unknown Location:',menu_edit.region.value_counts()['UNKNOWN'], "Number of Menus:", menu_edit.shape[0])

Number of Menus with Unknown Location: 12746 Number of Menus: 17547


## Clean the Dates for Dishes

In order to more easily work with the year column in the Menu table to correct and fill the first and last appeared dates in the Dishes table, I convert it to a datetime format where it exists, or a NaT format where year is unknown (or out of bounds).

In [12]:
menu_edit['year_edited'] = menu_edit['year']
menu_edit.loc[menu_edit.year_edited.str.lower()=='unknown', 'year_edited'] = '0'
menu_edit['year_edited'] = menu_edit['year_edited'].astype(int)
menu_edit.loc[(menu_edit.year_edited<1848)|(menu_edit.year_edited>2021), 'year_edited'] = np.nan
menu_edit['year_edited_dt'] = pd.to_datetime(menu_edit.year_edited, format='%Y')

In [13]:
print("Number of Menus missing years:", menu_edit['year_edited_dt'].isnull().sum())
print("Number of Menus:", menu_edit.shape[0])

Number of Menus missing years: 591
Number of Menus: 17547


In [14]:
menupage_df = pd.read_csv('MenuPage.csv')
mp_edit = menupage_df.copy()
menuitem_df = pd.read_csv('MenuItem.csv')
mi_edit = menuitem_df.copy()
dishes_df = pd.read_csv('NYPL-dishes-v2.csv')
dishes_edit = dishes_df.copy()

apply_date_fix() assign an NaT value to any incorrect dates in the Dishes table (either entries equal to '0', '1' or in the future and unable to be converted to date time). Otherwise, first and last appeared dates are converted to datetime.

In [15]:
def apply_date_fix(df):
    first_appeared = df['first_appeared']
    last_appeared = df['last_appeared']
    if first_appeared == '0' or first_appeared == '1':
        df['first_appeared_unknown'] = np.nan
    else:
        try:
            df['first_appeared_unknown'] = pd.to_datetime(first_appeared)
        except:
            df['first_appeared_unknown'] = np.nan
        
    if last_appeared == '0' or last_appeared == '1':
        df['last_appeared_unknown'] = np.nan
    else:
        try:
            df['last_appeared_unknown'] = pd.to_datetime(last_appeared)
        except:
            df['last_appeared_unknown'] = np.nan
    return df

In [16]:
dishes_edit['first_appeared_unknown'] = dishes_edit['first_appeared']
dishes_edit['last_appeared_unknown'] = dishes_edit['last_appeared']
dishes_edit = dishes_edit.apply(apply_date_fix, axis=1)

In [17]:
print("Number of dishes with invalid first_appeared date:",dishes_edit['first_appeared_unknown'].isnull().sum())
print("Number of dishes with invalid last_appeared date:",dishes_edit['last_appeared_unknown'].isnull().sum())
print("Total Number of Dishes:", dishes_edit.shape[0])

Number of dishes with invalid first_appeared date: 47490
Number of dishes with invalid last_appeared date: 47483
Total Number of Dishes: 343698


In order to fill entries in the Dishes table that have invalid first and/or last appeared dates with the correct ones, I first need to merge the Menu table, Menu Item table, Menu Pages table and Dishes table all together. Then, for each remaining Dish entry ID, I get the minimum and maximum non-Null years from the Menus it appears in.

In [18]:
master= mi_edit.merge(mp_edit.merge(menu_edit.rename({'id':'menu_id'}, axis='columns'), on='menu_id')\
                             .rename({'id':'menu_page_id'},axis='columns'), on='menu_page_id')\
                .merge(dishes_edit.rename({'id':'dish_id'},axis='columns'), on='dish_id').rename({'id':'menu_item_id'},axis='columns')

In [19]:
menu_dish_dt = master[['menu_id', 'dish_id', 'year_edited_dt', 'first_appeared_unknown', 'last_appeared_unknown']]
menu_dish_dt['dish_id'] = menu_dish_dt.dish_id.astype(int)
first_corrections = menu_dish_dt[~menu_dish_dt.year_edited_dt.isnull()].groupby('dish_id').year_edited_dt.min().to_dict()
last_corrections = menu_dish_dt[~menu_dish_dt.year_edited_dt.isnull()].groupby('dish_id').year_edited_dt.max().to_dict()

<ipython-input-19-7b265f959d7f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  menu_dish_dt['dish_id'] = menu_dish_dt.dish_id.astype(int)


Finally, I apply the date corrections where I can.

In [20]:
def apply_corrected_dt(df):
    if df['id'] in first_corrections:
        df['first_appeared_corrected'] = pd.to_datetime(first_corrections[df['id']],utc=True)
    if df['id'] in last_corrections:
        df['last_appeared_corrected'] = pd.to_datetime(last_corrections[df['id']],utc=True)
    return df

In [21]:
dishes_edit['first_appeared_corrected'] = dishes_edit['first_appeared_unknown']
dishes_edit['last_appeared_corrected'] = dishes_edit['last_appeared_unknown']
dishes_edit = dishes_edit.apply(apply_corrected_dt, axis='columns')

In [22]:
print("Number of dishes with corrected first_appeared:",(pd.DatetimeIndex(dishes_edit.first_appeared_corrected).year!=pd.DatetimeIndex(dishes_edit.first_appeared_unknown).year).sum())
print("Number of dishes with corrected last_appeared:",(pd.DatetimeIndex(dishes_edit.last_appeared_corrected).year!=pd.DatetimeIndex(dishes_edit.last_appeared_unknown).year).sum())
print("Number of Nulls corrected (first_appeared):", dishes_edit.first_appeared_unknown.isnull().sum() - dishes_edit.first_appeared_corrected.isnull().sum())
print("Number of Nulls corrected (last_appeared):", dishes_edit.last_appeared_unknown.isnull().sum() - dishes_edit.last_appeared_corrected.isnull().sum())
print("Total Dishes:", dishes_edit.shape[0])

Number of dishes with corrected first_appeared: 49079
Number of dishes with corrected last_appeared: 49020
Number of Nulls corrected (first_appeared): 11381
Number of Nulls corrected (last_appeared): 11375
Total Dishes: 343698


### IVC corrections

The main consideration here outside of what has been done so far is that it seems that there are dishes, menu items, menu pages, and menus that are only accounted for in their respective tables, but are not actually linked to each other. Because pandas by default performs inner joins, we can accomplish this IVC fairly easily by simply removing items in the dishes, menu items, menu pages and menu tables that do not have corresponding entries in the master merge table created above. We will also make sure each table only has one ID per entry.

In [23]:
menupage_ids = master.menu_page_id.astype(int).unique()
mp_edit = mp_edit[mp_edit.id.isin(menupage_ids)]
print("Number of Menu Page entries in original:", menupage_df.id.unique().shape[0])
print("Number of Menu Page entries in final:", mp_edit.id.unique().shape[0])
print("One ID per entry?", mp_edit.shape[0] == mp_edit.id.unique().shape[0])

Number of Menu Page entries in original: 66937
Number of Menu Page entries in final: 26443
One ID per entry? True


In [24]:
menuitem_ids = master.menu_item_id.astype(int).unique()
mi_edit = mi_edit[mi_edit.id.isin(menuitem_ids)]
print("Number of Menu Item entries in original:", menuitem_df.id.unique().shape[0])
print("Number of Menu Item entries in final:", mi_edit.id.unique().shape[0])
print("One ID per entry?", mi_edit.shape[0] == mi_edit.id.unique().shape[0])

Number of Menu Item entries in original: 1334784
Number of Menu Item entries in final: 1029756
One ID per entry? True


In [25]:
menu_ids = master.menu_id.astype(int).unique()
menu_edit = menu_edit[menu_edit.id.isin(menu_ids)]
print("Number of Menu entries in original:", menu_df.id.unique().shape[0])
print("Number of Menu entries in final:", menu_edit.id.unique().shape[0])
print("One ID per entry?", menu_edit.shape[0] == menu_edit.id.unique().shape[0])

Number of Menu entries in original: 17547
Number of Menu entries in final: 17504
One ID per entry? True


In [26]:
dish_ids = master.dish_id.astype(int).unique()
dishes_edit = dishes_edit[dishes_edit.id.isin(dish_ids)]
print("Number of Menu entries in original:", dishes_df.id.unique().shape[0])
print("Number of Menu entries in final:", dishes_edit.id.unique().shape[0])
print("One ID per entry?", dishes_edit.shape[0] == dishes_edit.id.unique().shape[0])

Number of Menu entries in original: 343698
Number of Menu entries in final: 331536
One ID per entry? True


In [27]:
mp_edit.to_csv('MenuPage_cleaned.csv', index=False)
mi_edit.to_csv('MenuItem_cleaned.csv', index=False)
menu_edit.to_csv('Menu_cleaned.csv', index=False)
dishes_edit.to_csv('Dish_cleaned.csv', index=False)